In [1]:
import os
import time
import serial
from tkinter import *
from tkinter import ttk, filedialog
from tkinter.filedialog import askopenfile, askdirectory, askopenfilename
from tkinter.scrolledtext import ScrolledText
from PIL import ImageTk, Image
#from general import *
# import FSOCRepeatCode
# import FSOCHammingCode
# import FSOCBCHCode
# import FSOCReedSolomonCode
import datetime
import serial.tools.list_ports

In [2]:
modlist = ["OOK", "PPM", "DPPM", "DPIM", "DHPIM"]
feclist = [
    "<none>",
    "Repeat Code 03",
    "Repeat Code 05",
    "Hamming Code",
    "LDPC Code",
    "BCH Code",
    "Reed Solomon Code",
]
fecconvlist = [1, 3, 5, 2, 2, 4, 6]
baudlist = [110, 300, 600, 1200, 2400, 4800, 9600, 14400, 19200, 38400, 57600, 115200, 128000, 230400, 460800, 576000, 921600, 1000000 ,2000000,3000000]
TCOMPortlist= ['COM1','COM2', 'COM3']
RCOMPortlist= ['COM1','COM2', 'COM3','COM4']

In [3]:
ports = serial.tools.list_ports.comports()

comportdesc= []
comportlist=[]
for port, desc, hwid in sorted(ports):
        comportdesc.append(desc)
        comportlist.append(port)


In [4]:
def errorLog(data):
    """
    Updates the contents of a label widget named labelLog with the error message passed as input
    
    Arguments:
    data  -- Error message
   
    """

    # Set the label to be editable
    labelLog.configure(state='normal')

    # Insert the error message into the label
    labelLog.insert(INSERT, "\tERROR: " + data + "\n")

    # Add a tag to highlight the error message and position defined by a range delimited by the positions startindex and endindex
    labelLog.tag_add("start", "end-2l", "end-1l")

    # Configure the tag to have text in red and bold
    labelLog.tag_config("start",
                        foreground="red",
                        font=("Myriad Pro Light", 10, 'bold'))

    # Set the label to be uneditable
    # Otherwise backspace can clear the content in the Log
    labelLog.configure(state='disabled')


def appendLog(data):
    """
    Arguments:
    data -- The text to be appended to the label widget   
    
    """

    # Set the label to be editable
    labelLog.configure(state='normal')

    # Inserts the given data to the label widget at the current insertion cursor position,
    #print(data, end='')
    labelLog.insert(INSERT, data)

    # Set the label to be uneditable
    labelLog.configure(state='disabled')


# Used to select any required file from file explorer and
def browseDir():
    """
    Used to browse and select any required file from file explorer and deletes the previous file path and allows to select a new file  
    
    """

    # Deletes the previous file path
    fileDir.delete(0, END)

    # Opens the file explorer to select a file
    tfFileDir = filedialog.askopenfilename()

    # Inserts the selected file path in the Entry widget
    fileDir.insert(0, tfFileDir)


def setzero():
    # Deletes the content of the Entry widget
    fileDir.delete(0, 'end')

COMports ="COM3"

def transferFn():
    dir = fileDir.get()
    port = comportlist[comportdesc.index(TCOMPort.get())]
   
    # Check if the file directory is empty
    if (dir == "") or (dir == " "):

        # If file directory is empty, display an error message in the log
        errorLog("Please Enter File Directory")

    else:
        #Update the log with the selected file directory and the modulation and forward error correction codes selected by the user
        data = "File: " + dir + "\n"
        labelLog.configure(state='normal')
        labelLog.delete("1.0", "end-1c")
        labelLog.insert(INSERT, data)
        labelLog.configure(state='disabled')
        appendLog("\nModulation Technique :" + mod.get())
        appendLog("\nForward Error Correction Code :" + fec.get())
        appendLog("\nBaudrate :"+ str(baud.get()))
        appendLog("\nCOMport :"+TCOMPort.get())
        #appendLog("\nExtra Forward Error Correction Code :" + fecEx.get())
        appendLog("\n\nTransferring...")
#         print(modlist.index(mod.get()))

        # The transmission function will transmit the data now
        transmit(dir, mod = modlist.index(mod.get()), fec = feclist.index(fec.get()), port = port, mainBaudrate = baud.get())


        #setzero()
def receiveFn():

    # Enable the label for editing
    labelLog.configure(state='normal')

    # Clear the contents of the label from line 2 onwards
    labelLog.delete("2.0", 'end')

    # Disable the label for editing
    labelLog.configure(state='disabled')
    
    port = comportlist[comportdesc.index(RCOMPort.get())]
    
    # Call the appendLog function to add a new line to the label
    appendLog("\nReceiving...")

    # Call the receive function
    receive(port)



In [5]:
def receive(port="COM3"
            ):  # Default port is set for Raspberry Pi's serial port

    # Create a serial object for metadata using the specified port and reliable baud rate
    # As meta data is about file information, It should be sent with reliable communication parameters
    serMeta = serial.Serial(
        port=port,
        baudrate=
        57600,  # Thus baud rate should be reduced if mainbaud rate goes below it #57600
        bytesize=8,
        timeout=2,
        stopbits=serial.STOPBITS_ONE)

    # Keep reading from the serial port until the "finish\n" string is received
    while True:
        line = serMeta.readline()
        if (line == b'finish\n'):

            # Close the serial meta port
            serMeta.close()
            break

        metadata_bytes = line

    # Get the current date and time as a string in the format "%y%m%d_%H%M%S" for file log
    # In commercial scale, it is not required.
    # If you are commenting the time, then uncomment the below line to avoid errors
    #dt_string = ''
    #now = datetime.now()
    #dt_string = now.strftime("%y%m%d_%H%M%S ")
    #print("date and time =", dt_string)

    # Record the start time using the time.perf_counter() function
    sttime = time.perf_counter()

    # Decode the received bytes as a string
    metadata = metadata_bytes.decode('latin1')

    # Split the metadata string into an list using "|||" as the separator
    metadata_array = metadata.split("|||")
    # Refer Chapter 3.2.1 for metadata structure

    # 1st element
    # Get the main communication baud rate from the metadata list | Covert it to integer from string
    mainBaudRateRx = int(metadata_array[0])

    # 2nd element
    # Combine the date and time string with the second element of the metadata list to form the filename
    # Note when you dont need dt_string, just comment the datetime portion in this program
    # and uncomment the following line:
    #dt_string = ''
    #filename = dt_string + metadata_array[1]
    filename = metadata_array[1]

    # 3rd element
    # Get the length of the file from the metadata list | Convert it to integer
    length_of_file = int(metadata_array[2])

    # 4th element
    # Get the modulation scheme used from the metadata list
    #use it as an index to retrieve the corresponding value from the modlist array
    mod = modlist[int(metadata_array[3])]

    # 5th element
    # Get the FEC used from the metadata list and convert to an integer
    fec = int(metadata_array[4])

    # 6th element
    # Get the Extra FEC used from the metadata list and convert to an integer
    # fecEx = int(metadata_array[5])

    #After Receiving Meta'finish', it took 0.00020s(20us) for these operations
    # Record the end time using the time.perf_counter() function
    endtime = time.perf_counter()

    # Call the appendLog function to add a new line to the log
    appendLog("\nMetadata received\n")

    # Call the appendLog function to add the metadata array to the log
    appendLog(metadata_array)

    # Create a new serial object for main communication using the mainBaudRateRx
    serMain = serial.Serial(port=port,
                            baudrate=mainBaudRateRx,
                            bytesize=8,
                            timeout=0.1,
                            stopbits=serial.STOPBITS_ONE)

    # Determine the number of bits to read based on the fec and fecEx values
    if (fec == 2):
        bitread = 30
    else:
        bitread = 48

    # Log the bit read
    appendLog(bitread)

    # Initialize variables for counting bits, tracking previous bits, and detecting the start of a communication
    count = 0
    prevbit = b''
    commstart = False

    # Open the file to write received data and clear it
    fh = open("Received/" + filename, "ab")
    fh.truncate(0)

    # Open the file to write fec received data and clear it
    # fh_fec = open(
    #     "Files/ReceivedFiles/test/" + filename.rsplit(".", 1)[0] +
    #     metadata_array[3] + metadata_array[4] + metadata_array[5] + "RX.txt",
    #     "wb")
    # fh_fec.truncate(0)

    # Continuous Loop
    while True:

        # Read bits from the main serial connection
        txtbit = serMain.read(bitread)

        # Save the original bits before applying FEC
        txtbit_fec = txtbit
        #print(len(txtbit),txtbit)

        # Apply the FEC code for error correction
#         if (fecEx == 0):
#             pass
#         elif (fecEx == 1):
#             txtbit = FSOCRepeatCode.r03dec(txtbit)
#         elif (fecEx == 2):
#             txtbit = FSOCRepeatCode.r05dec(txtbit)
#         elif (fecEx == 3):
#             txtbit = FSOCHammingCode.hamdec(txtbit)
#         elif (fecEx == 4):
#             txtbit = FSOCLDPCCode.ldpcdec(txtbit)
#         elif (fecEx == 5):
#             txtbit = FSOCBCHCode.bchdec(txtbit)
#         elif (fecEx == 6):
#             txtbit = FSOCReedSolomonCode.rsdec(txtbit)
#         else:
#             raise ValueError(f'FEC extra is not in the list')

        # Apply the FEC code for error correction again if needed
        if (fec == 0):
            pass
        elif (fec == 1):
            txtbit = FSOCRepeatCode.r03dec(txtbit)
        elif (fec == 2):
            txtbit = FSOCRepeatCode.r05dec(txtbit)
        elif (fec == 3):
            txtbit = FSOCHammingCode.hamdec(txtbit)
        elif (fec == 4):
            txtbit = FSOCLDPCCode.ldpcdec(txtbit)
        elif (fec == 5):
            txtbit = FSOCBCHCode.bchdec(txtbit)
        elif (fec == 6):
            txtbit = FSOCReedSolomonCode.rsdec(txtbit)
        else:
            raise ValueError(f'FEC is not in the list')

        # Check if the current bit stream is different from the previous bit stream and if it is the first bit stream
        if (txtbit != prevbit) & (prevbit == b'') & (not commstart):

            # Set communication start flag to True
            commstart = True

            # Append "Communication Started" to the log file
            appendLog("\nCommunication Started")

            # Save the start time
            starttime = time.perf_counter()

        # Write the bits to the output file and increase the count
        if commstart:
            #if count%50 == 0:
            #    print(count)
            fh.write(txtbit)
            #print(txtbit_fec, len(txtbit_fec))
            #print(txtbit, len(txtbit))
            count += 1

            # Check if the current bit stream is empty
            if txtbit == b'':

                # Save the end time
                endtime = time.perf_counter()

                # Adjust the count based on the previous bit stream length
                count -= (2 - (len(prevbit) / (bitread * 1)))

                # Append "Stopped Reading Serial Bits" to the log file
                appendLog("\nStopped Reading Serial Bits")

                # Set communication start flag to False
                commstart = False

                # Close the output files
                fh.close()
                break

        # Save the current bit stream as the previous bit stream
        prevbit = txtbit
    serMain.close()
    difftime = endtime - starttime
    appendLog("\nrecd code executed in - " + str(formattime(difftime)) + "@" +
              str(round(count * bitread / (difftime) / 1000, 2)) + "KBps")

In [6]:

def transmit(directory,
             mod=0,
             fec=0,
             port="/dev/ttyS0",
             mainBaudrate=57600):            #57600

    # Read file contents in binary mode
    with open(directory, "rb") as text_file:
        text_data = text_file.read()

    # Apply Forward Error Correction (FEC) if specified
    if (fec == 1):
        text_data = FSOCRepeatCode.r03enc(text_data)
    elif (fec == 2):
        text_data = FSOCRepeatCode.r05enc(text_data)
    elif (fec == 3):
        text_data = FSOCHammingCode.hamenc(text_data)
    elif (fec == 4):
        text_data = FSOCLDPCCode.ldpcenc(text_data)
    elif (fec == 5):
        text_data = FSOCBCHCode.bchenc(text_data)
    elif (fec == 6):
        text_data = FSOCReedSolomonCode.rsenc(text_data)
    elif (fec == 0):
        text_data = text_data
    else:

        # Raise exception if invalid FEC code is provided
        raise ValueError(
            f"\nUnknown Error Correction Code Referred. Valid Code Numbers are [ 0 to 6 ]. \nCode Number : {fec}"
        )

    # Apply Extra Forward Error Correction (FEC) if specified
#     if (fecEx == 1):
#         text_data = FSOCRepeatCode.r03enc(text_data)
#     elif (fecEx == 2):
#         text_data = FSOCRepeatCode.r05enc(text_data)
#     elif (fecEx == 3):
#         text_data = FSOCHammingCode.hamenc(text_data)
#     elif (fecEx == 4):
#         text_data = FSOCLDPCCode.ldpcenc(text_data)
#     elif (fecEx == 5):
#         text_data = FSOCBCHCode.bchenc(text_data)
#     elif (fecEx == 6):
#         text_data = FSOCReedSolomonCode.rsenc(text_data)
#     elif (fecEx == 0):
#         text_data = text_data
#     else:

#         # Raise exception if invalid FEC code is provided
#         raise ValueError(
#             f"\nUnknown Error Correction Code Referred. Valid Code Numbers are [ 0 to 6 ]. \nCode Number : {fecEx}"
#         )

    # Create a new file name with modulation technique and FEC codes used
    file = os.path.split(directory)[1].rsplit(
        '.', 1)[0] + str(mod) + str(fec) + "TX.txt"
    directory_fec = os.path.join(os.path.split(directory)[0], file)

    # Write modified data to new file in binary mode
    with open(directory_fec, "wb") as text_file_fec:
        # print(directory_fec)
        text_file_fec.write(text_data)

    # Calculate length of data
    length_of_file = len(text_data)
    file = os.path.split(directory)[1]

    # Create metadata for the transmission
    sep = "|||"
    metadata = str(mainBaudrate) + sep + file + sep + str(
        length_of_file) + sep + str(mod) + sep + str(fec) + sep + "\nfinish\n"
    metadata_bytes = metadata.encode('latin1')
    appendLog("\n" + metadata)

    # Open a serial connection with the specified port and settings for metadata transmission
    serMeta = serial.Serial(port=port,
                            baudrate=57600,
                            bytesize=8,
                            timeout=2,
                            stopbits=serial.STOPBITS_ONE)

    # Write the metadata to the serial port
    serMeta.write(metadata_bytes)

    # Wait for 100ms to allow time for the metadata to be sent
    time.sleep(0.1)

    # Close the metadata serial connection
    serMeta.close()

    # Log that metadata transmission was successful
    appendLog("\nMetadata transmitted")

    # Wait for 1 second before opening the main serial connection
    time.sleep(1)

    # Open a serial connection with the specified port and settings for main data transmission
    serMain = serial.Serial(port=port,
                            baudrate=mainBaudrate,
                            bytesize=8,
                            timeout=2,
                            stopbits=serial.STOPBITS_ONE)

    # Get the start time for measuring the time taken for transmitting the data
    starttime = time.perf_counter()

    # Write the text data to the serial port
    serMain.write(text_data)
    #print(text_data)

    # Get the end time for measuring the time taken for transmitting the data
    endtime = time.perf_counter()

    # Log that text data transmission was successful and report the time taken for transmission
    appendLog("\ntxmtd successfully in " +
              str(formattime(endtime - starttime)))

    # Close the main serial connection
    serMain.close()

    # Log that data transmission code execution is complete
    appendLog("\ntxmtd code executed")

In [7]:
testColor=['silver','blue','green','orange','black','violet','pink','indigo','yellow','red','tan']
testPad=5
expandVar=1

In [8]:
bgColor = []
for _ in testColor:
    bgColor.append('#ffffff')
mainColor = bgColor
dropwidth = 25
#dpcolor='#ffffff'
dpcolor='#ffffff'

path = "sig.png"

In [9]:
root = Tk()
root.geometry("1250x760")
root.minsize(1250, 760)
root.maxsize(1250, 760)
root.title("File Transfer Graphic User Interface")

Window = Frame(root, bg='black')
Window.pack(expand=1, side=TOP, fill=BOTH, padx=10, pady=(5,10))

frame1 = Frame(Window, bg=mainColor[0])  # Title frame
frame1.pack(expand=0, side=TOP, fill=X, anchor='n')

frame11 = Frame(frame1, bg=mainColor[1])  # Title frame
frame11.pack(expand=0, side=BOTTOM, fill=X, anchor='c')

frame2 = Frame(Window, bg=mainColor[2])  # Content frame
frame2.pack(expand=expandVar, side=TOP, fill=BOTH, padx=testPad, pady=(testPad,testPad*0.5))

frame3 = Frame(Window, bg=mainColor[5])  # Developer frame
frame3.pack(expand=0, side=TOP, fill=BOTH, padx=testPad, pady=(testPad*0.5,testPad))

frame31 = Frame(frame3, bg=mainColor[2])  # Logo frame
frame31.pack(expand=0, side=TOP, anchor='center')

frame32 = Frame(frame3, bg=mainColor[2])  # Developer frame
frame32.pack(expand=0, side=TOP, fill=BOTH)

frame21 = Frame(frame2, bg=mainColor[6])  # Botton frame
frame21.pack(expand=expandVar,
             side=LEFT,
             fill=BOTH,
             padx=testPad,
             pady=testPad)

frame22 = Frame(frame2, bg=mainColor[7])  # Log frame
frame22.pack(expand=expandVar,
             side=LEFT,
             fill=BOTH,
             padx=testPad,
             pady=testPad)

 #frame221 = Frame(frame22, bg=mainColor[7])   #For_Future use
 #frame221.pack(expand=expandVar,
  #         fill=BOTH,
   #         padx=testPad,
    #       pady=testPad)

frame211 = Frame(frame21, bg=mainColor[3])  # Select_file frame
frame211.pack(expand=0, side=TOP, fill=BOTH, padx=testPad, pady=testPad)

frame212 = Frame(frame21, bg=mainColor[4])  # Technical frame
frame212.pack(expand=expandVar,
              side=TOP,
              fill=BOTH,
              padx=testPad,
              pady=testPad)

frame321 = Frame(frame32, bg=mainColor[6]) 
frame321.pack(expand=1, side=LEFT, fill=BOTH)

frame322 = Frame(frame32, bg=mainColor[1])  
frame322.pack(expand=0, side=LEFT, fill=BOTH)


frame2111 = Frame(frame211, bg=mainColor[7])  #Browse_botton frame
frame2111.pack(expand=expandVar,
               side=BOTTOM,
               fill=BOTH,
               padx=testPad,
               pady=testPad)

frame2121 = Frame(frame212, bg=mainColor[1])
frame2121.pack(expand=expandVar,
               side=LEFT,
               fill=BOTH,
               padx=testPad,
               pady=testPad)

frame2122 = Frame(frame212, bg=mainColor[8])
frame2122.pack(expand=expandVar,
               side=LEFT,
               fill=BOTH,
               padx=testPad,
               pady=testPad)

frame21211 = Frame(frame2121, bg=mainColor[3])  # Basics frame
frame21211.pack(expand=0, side=TOP, fill=BOTH, padx=testPad, pady=testPad)

frame21212 = Frame(frame2121, bg=mainColor[7])  #left-right
frame21212.pack(expand=expandVar,
                side=TOP,
                fill=BOTH,
                padx=testPad,
                pady=testPad)

frame21221 = Frame(frame2122, bg=mainColor[6])  #For_Future use
frame21221.pack(expand=expandVar,
                side=RIGHT,
                fill=BOTH,
                padx=testPad,
                pady=testPad)

frame21222 = Frame(frame2122, bg=mainColor[7])  #right-left
frame21222.pack(expand=expandVar,
                side=LEFT,
                fill=BOTH,
                padx=testPad,
                pady=testPad)

frame212111 = Frame(frame21211, bg=mainColor[3])  # Modulation frame
frame212111.pack(expand=0, side=BOTTOM, fill=BOTH, padx=testPad, pady=testPad)

# frame212121= Frame(frame21212, bg=mainColor[0])  #For_Future use
# frame212121.pack(expand=0,
#               side=TOP,
#               fill=BOTH,
#               padx=testPad,
#               pady=testPad)

frame212122 = Frame(frame21212, bg=mainColor[1])
frame212122.pack(expand=expandVar,
                 side=BOTTOM,
                 fill=BOTH,
                 padx=testPad,
                 pady=testPad)

frame212123 = Frame(frame21212, bg=mainColor[2])
frame212123.pack(expand=0, side=BOTTOM, fill=BOTH, padx=testPad, pady=testPad)

frame2121221 = Frame(frame212122, bg=mainColor[9])
frame2121221.pack(expand=0, side=TOP, fill=BOTH, padx=testPad, pady=testPad)

frame2121222 = Frame(frame212122, bg=mainColor[0])  # Fec frame
frame2121222.pack(expand=0, side=TOP, fill=BOTH, padx=testPad, pady=0)

# frame2121231= Frame(frame212123, bg=mainColor[6])  #For_Future use
# frame2121231.pack(expand=0,
#               side=TOP,
#               fill=BOTH,
#               padx=testPad,
#               pady=testPad)

# frame2121232= Frame(frame212123, bg=mainColor[3]) # Fecx frame
# frame2121232.pack(expand=0,
#               side=TOP,
#               fill=BOTH,
#               padx=testPad,
#               pady=testPad)

frame212221 = Frame(frame21222, bg=mainColor[2])
frame212221.pack(expand=0, side=TOP, fill=BOTH, padx=testPad, pady=testPad)

frame212222 = Frame(frame21222, bg=mainColor[2])
frame212222.pack(expand=expandVar,
                 side=TOP,
                 fill=BOTH,
                 padx=testPad,
                 pady=testPad)

# frame2122211= Frame(frame212221, bg=mainColor[0]) #For_Future use
# frame2122211.pack(expand=0,
#               side=TOP,
#               fill=BOTH,
#               padx=testPad,
#               pady=testPad)

frame2122212 = Frame(frame212221, bg=mainColor[3])
frame2122212.pack(expand=0, side=TOP, fill=BOTH, padx=testPad, pady=testPad)

frame2122221 = Frame(frame212222, bg=mainColor[6])
frame2122221.pack(expand=0, side=TOP, fill=BOTH, padx=testPad, pady=testPad)

frame2122222 = Frame(frame212222, bg=mainColor[3])
frame2122222.pack(expand=0, side=TOP, fill=BOTH, padx=testPad, pady=testPad)

frame2122223 = Frame(frame212222, bg=mainColor[9])
frame2122223.pack(expand=expandVar,
                  side=TOP,
                  fill=BOTH,
                  padx=testPad,
                  pady=testPad)

frame21222231 = Frame(frame2122223, bg=mainColor[4])
frame21222231.pack(expand=expandVar,
                   side=BOTTOM,
                   fill=BOTH,
                   padx=testPad,
                   pady=testPad)

titlecard = Label(frame1,
                  text="Optical Wireless Communication File Transfer System",
                  font=("Calibri", 30, "bold"),
                  bg=mainColor[0])
titlecard.pack(anchor="n")

titlecard1 = Label(frame11,
                   text="(Free-Space,Fog Atmosphere ,Smoke Atmosphere and Long Distance Communication)",
                   font=("Calibri", 15, "bold"),
                   bg=mainColor[2])
titlecard1.pack(anchor="c")

labelSelectFile = Label(frame211,
                        text="Select The File :",
                        font=("Myriad Pro Light", 13),
                        bg=mainColor[3])
labelSelectFile.pack(side=BOTTOM, anchor='nw')

fileDir = Entry(frame2111, font=("Myriad Pro Bold", 10), width=45, bd=0)
fileDir.pack(side=LEFT, anchor='nw', ipady=5.6)

browseBtn = Button(
    frame2111,
    font=("Myriad Pro Light", 10),
    text="Browse File",
    bd=0,
    bg="#dda15e",
    command=browseDir,
)

Label(
    frame212221,
    text="Transmitter :",
    font=("Myriad Pro", 13),
    bg=mainColor[3],
).pack(side=LEFT, anchor='nw', fill="x")

browseBtn.pack(expand=0, side=LEFT, anchor='nw', ipady=3)

Label(frame21211, text="Basics :", font=("Myriad Pro", 15),
      bg=mainColor[0]).pack(side=LEFT, fill="x", anchor='nw')

#Label(frame212111,
 #     text="Modulation Techniques :",
  #    font=("Myriad Pro", 10),
   #   bg=mainColor[5]).pack(anchor='w', pady=(10, 0))

#mod = StringVar()
#mod.set(modlist[0])

#modOptions = OptionMenu(frame212111, mod, *modlist)
#modOptions.config(bg=dpcolor, relief="flat", width=dropwidth)
#modOptions["menu"].config(bg=mainColor[3])
#modOptions.pack(anchor='nw', padx=1.6, pady=(0, 10))

Label(frame21212, text="Error Correction Technique :", font=("Myriad Pro", 13),
      bg=mainColor[9]).pack(side=TOP, anchor='nw')

fec = StringVar()
fec.set(feclist[0])

Label(
    frame2121221,
    text="Forward Error Correction Code :",
    font=("Myriad Pro", 10),
    bg=mainColor[3],
).pack(side=LEFT, anchor='nw', fill="x")

fecOptions = OptionMenu(frame2121222, fec, *feclist)
fecOptions.config(bg=dpcolor, relief="flat", width=dropwidth)
fecOptions["menu"].config(bg=mainColor[6])
fecOptions.pack(anchor=W, padx=10, pady=(0, 10))


baud = IntVar()
baud.set(baudlist[0])

Label(frame212111,
      text="Baud Rate :",
      font=("Myriad Pro", 10),
      bg=mainColor[7]).pack(side=TOP, anchor='nw', pady=(10, 0))

baudOptions = OptionMenu(frame212111, baud, *baudlist)
baudOptions.config(bg=dpcolor, relief="flat", width=dropwidth)
baudOptions["menu"].config(bg=mainColor[2])
baudOptions.pack(side=LEFT, anchor='n', pady=(0, 10))

TCOMPort = StringVar()
TCOMPort.set("--Select the Port--")

Label(frame2122221,
      text="TCOMPort :",
      font=("Myriad Pro", 10),
      bg=mainColor[0]).pack(side=LEFT, anchor='nw', fill="x")

TCOMPortOptions = OptionMenu(frame2122222, TCOMPort, *comportdesc)
TCOMPortOptions.config(bg=dpcolor, relief="flat", width=dropwidth)
TCOMPortOptions["menu"].config(bg=mainColor[0])
TCOMPortOptions.pack(anchor='nw', padx=1.6, pady=(0, 10))

transferBtn = Button(
    frame2122223,
    font=("Myriad Pro", 12, "bold"),
    text="Send >> ",
    relief="groove",
    bg="#283618",
    fg="#fff2f2",
    command=transferFn,
)
transferBtn.pack(side=TOP, ipadx=11, ipady=4, anchor="center")

Label(
    frame2122223,
    text="Receiver :",
    font=("Myriad Pro", 13),
    bg=mainColor[3],
).pack(side=LEFT, anchor='nw', fill="x")

RCOMPort = StringVar()
RCOMPort.set('--Select the Port--')

Label(frame21222231,
      text="RCOMPort :",
      font=("Myriad Pro", 10),
      bg=mainColor[0]).pack(side=TOP, pady=(10, 0), anchor='nw')

RCOMPortOptions = OptionMenu(frame21222231, RCOMPort, *comportdesc)
RCOMPortOptions.config(bg=dpcolor, relief="flat", width=dropwidth)
RCOMPortOptions["menu"].config(bg=mainColor[0])
RCOMPortOptions.pack(anchor='nw', padx=1.6, pady=(0, 10))

receiverBtn = Button(
    frame21222231,
    font=("Myriad Pro", 12, "bold"),
    text="Receive  << ",
    relief="groove",
    bg="#606c38",
    fg="#fff2f2",
    command=receiveFn,
)
receiverBtn.pack(side=TOP, ipadx=11, ipady=4, anchor="center")

Label(frame22, text="Log :", font=("Myriad Pro", 10),
      bg=mainColor[2]).pack(side=TOP, anchor=W)
labelLog = ScrolledText(
    frame22,
    relief="groove",
    state="disabled",
    #width=30, height=5,
    font=("Myriad Pro Light", 10),
    bg=dpcolor,
)  #'#FEFBE9'
mod = StringVar()
mod.set(modlist[0])
labelLog.pack(expand=2, side=BOTTOM, fill=BOTH, anchor="n")

labelLog.config(state="normal", spacing3=2)
labelLog.insert(INSERT, "Browse the File...\n")
labelLog.config(state="disabled")

DIAT_Logo = Image.open("sig.png", mode='r')
DIAT_Logo_resize = DIAT_Logo.resize((50, 50))
DIAT_LogoImg = ImageTk.PhotoImage(DIAT_Logo_resize)
labelLogo = Label(frame31, image=DIAT_LogoImg, bg = mainColor[2])
labelLogo.image = DIAT_LogoImg
labelLogo.pack(side=LEFT, anchor='center')



Label(
    frame31,
    text="SINHGAD COLLEGE OF ENGINEERING ,PUNE",
    font=("Times New Roman", 18,'bold'),
    bg=mainColor[7],
).pack(side=TOP, anchor='center',fill=BOTH,expand=1)

Label(
    frame321,
    text="Designed And Developed By:",
    font=("Myriad Pro", 10,'italic'),
    bg=mainColor[3],
).pack(side=TOP, anchor='sw')

Label(
    frame321,
    text=" SINHGAD COLLEGE OF ENGINEERING ,PUNE",
    font=("Myriad Pro", 15),
    bg=mainColor[3],
).pack(side=TOP, anchor='sw')

Label(
    frame322,
    text="Contact:",
    font=("Myriad Pro", 10,'italic'),
    bg=mainColor[3],
).pack(side=TOP, anchor='sw')

Label(
    frame322,
    text="Dr. AVR Murthy, Ishwar Mundhe, Nidhi Mishra , Khushi Nigachi",
    font=("Myriad Pro", 12),
    bg=mainColor[3],
).pack(side=TOP, anchor='se')





root.mainloop()

2024-01-18 09:36:58.758 Python[51237:11906283] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
